In [1]:
import os

In [2]:
os.getcwd()

'C:\\Users\\ANUSHA\\Desktop'

In [3]:
os.chdir(r"C:\Users\ANUSHA\Desktop\project\dataset")

In [4]:
import nltk
import re
import numpy as np # linear algebra
import pandas as pd # data processing
import random
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import SnowballStemmer
stemmer=SnowballStemmer('english')

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.tokenize import word_tokenize


#import pandas_profiling

%matplotlib inline

In [5]:
data=pd.read_csv("reviews.csv")

In [6]:
data

,id,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,B000F83SZQ,"[0, 0]",5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J,Avidreader,Nice vintage story,1399248000
1,1,B000F83SZQ,"[2, 2]",4,This book is a reissue of an old one; the auth...,"01 6, 2014",AN0N05A9LIJEQ,critters,Different...,1388966400
2,2,B000F83SZQ,"[2, 2]",4,This was a fairly interesting read. It had ol...,"04 4, 2014",A795DMNCJILA6,dot,Oldie,1396569600
3,3,B000F83SZQ,"[1, 1]",5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",A1FV0SX13TWVXQ,"Elaine H. Turley ""Montana Songbird""",I really liked it.,1392768000
4,4,B000F83SZQ,"[0, 1]",4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",A3SPTOKDG7WBLN,Father Dowling Fan,Period Mystery,1395187200
...,...,...,...,...,...,...,...,...,...,...
1494,1494,B0017Y914Y,"[2, 4]",2,Why ruin a good book with sex scenes?This was ...,"01 16, 2013",A3J5CM842V51NC,Laurel,"Good, except for the sex",1358294400
1495,1495,B0017Y914Y,"[1, 1]",5,"At first, I didn't know what to expect from th...","05 1, 2014",A3MBL5GPPLVX5M,luvmykindle,Hard to believe that this is a true story,1398902400
1496,1496,B0017Y914Y,"[2, 2]",5,The Remnants: Stories of the Jewish Resistance...,"06 19, 2012",A2BYK88GD8AX1C,"Melissa A. Tackett ""Dark Lover""",No The Jews Did not go quietly to Their Death,1340064000
1497,1497,B0017Y914Y,"[17, 30]",1,I admit that at first I thought I was reading ...,"01 28, 2012",A1I6L9YZTSDHPM,meme,Disappointed,1327708800


In [7]:
data.head()

,id,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,B000F83SZQ,"[0, 0]",5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J,Avidreader,Nice vintage story,1399248000
1,1,B000F83SZQ,"[2, 2]",4,This book is a reissue of an old one; the auth...,"01 6, 2014",AN0N05A9LIJEQ,critters,Different...,1388966400
2,2,B000F83SZQ,"[2, 2]",4,This was a fairly interesting read. It had ol...,"04 4, 2014",A795DMNCJILA6,dot,Oldie,1396569600
3,3,B000F83SZQ,"[1, 1]",5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",A1FV0SX13TWVXQ,"Elaine H. Turley ""Montana Songbird""",I really liked it.,1392768000
4,4,B000F83SZQ,"[0, 1]",4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",A3SPTOKDG7WBLN,Father Dowling Fan,Period Mystery,1395187200


In [8]:
df=data

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 10 columns):
id                1499 non-null int64
asin              1499 non-null object
helpful           1499 non-null object
overall           1499 non-null int64
reviewText        1499 non-null object
reviewTime        1499 non-null object
reviewerID        1499 non-null object
reviewerName      1495 non-null object
summary           1499 non-null object
unixReviewTime    1499 non-null int64
dtypes: int64(3), object(7)
memory usage: 117.2+ KB


In [11]:
df.describe()

,id,overall,unixReviewTime
count,1499.00000,1499.00000,1.499000e+03
mean,749.00000,4.12008,1.344308e+09
std,432.86834,1.10720,5.426525e+07
min,0.00000,1.00000,1.036454e+09
25%,374.50000,4.00000,1.312070e+09
50%,749.00000,4.00000,1.361578e+09
75%,1123.50000,5.00000,1.386418e+09
max,1498.00000,5.00000,1.405555e+09


In [13]:
df=df[['reviewText','overall']]
df.head()

,reviewText,overall
0,I enjoy vintage books and movies so I enjoyed ...,5
1,This book is a reissue of an old one; the auth...,4
2,This was a fairly interesting read. It had ol...,4
3,I'd never read any of the Amy Brewster mysteri...,5
4,"If you like period pieces - clothing, lingo, y...",4


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 2 columns):
reviewText    1499 non-null object
overall       1499 non-null int64
dtypes: int64(1), object(1)
memory usage: 23.5+ KB


In [15]:
df=df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1499 entries, 0 to 1498
Data columns (total 2 columns):
reviewText    1499 non-null object
overall       1499 non-null int64
dtypes: int64(1), object(1)
memory usage: 35.1+ KB


In [16]:
df=df[df['overall']!=3]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1296 entries, 0 to 1498
Data columns (total 2 columns):
reviewText    1296 non-null object
overall       1296 non-null int64
dtypes: int64(1), object(1)
memory usage: 30.4+ KB


In [17]:
df=df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296 entries, 0 to 1295
Data columns (total 2 columns):
reviewText    1296 non-null object
overall       1296 non-null int64
dtypes: int64(1), object(1)
memory usage: 20.4+ KB


In [18]:
df['sentiment']=np.where(df['overall']>3,1,0)
df.head()

,reviewText,overall,sentiment
0,I enjoy vintage books and movies so I enjoyed ...,5,1
1,This book is a reissue of an old one; the auth...,4,1
2,This was a fairly interesting read. It had ol...,4,1
3,I'd never read any of the Amy Brewster mysteri...,5,1
4,"If you like period pieces - clothing, lingo, y...",4,1


In [19]:
df.tail()

,reviewText,overall,sentiment
1291,Why ruin a good book with sex scenes?This was ...,2,0
1292,"At first, I didn't know what to expect from th...",5,1
1293,The Remnants: Stories of the Jewish Resistance...,5,1
1294,I admit that at first I thought I was reading ...,1,0
1295,"No the usual holocaust book, although they are...",5,1


In [20]:
df

,reviewText,overall,sentiment
0,I enjoy vintage books and movies so I enjoyed ...,5,1
1,This book is a reissue of an old one; the auth...,4,1
2,This was a fairly interesting read. It had ol...,4,1
3,I'd never read any of the Amy Brewster mysteri...,5,1
4,"If you like period pieces - clothing, lingo, y...",4,1
...,...,...,...
1291,Why ruin a good book with sex scenes?This was ...,2,0
1292,"At first, I didn't know what to expect from th...",5,1
1293,The Remnants: Stories of the Jewish Resistance...,5,1
1294,I admit that at first I thought I was reading ...,1,0


In [21]:
df['sentiment'].mean()

0.8873456790123457

In [24]:
from  sklearn.model_selection import train_test_split

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['reviewText'],df['sentiment'], random_state=0)

In [25]:
X_train.iloc[1]

'I would recommend if you are trying to tie knots. All of the common knots are in here and some more unusual.'

In [26]:
X_train.shape

(972,)

In [27]:
y_train.shape

(972,)

In [28]:
from  sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from  sklearn.metrics import roc_auc_score

In [29]:
cv=CountVectorizer().fit(X_train)

In [30]:
X_train_vectorized = cv.transform(X_train)

In [31]:
# Train the model
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

C:\Users\ANUSHA\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [33]:
predictions = model.predict(cv.transform(X_test))

In [36]:
predictions

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,

In [54]:
r=roc_auc_score(y_test, predictions)
r

0.6026889597517883

In [50]:
feature_names = np.array(cv.get_feature_names())

In [51]:

feature_names

array(['00', '000', '0oh', ..., 'zombies', 'zone', 'zulu'], dtype='<U31')

In [57]:
r=model.predict(cv.transform([" "]))

In [58]:
r=r<0.6
r

array([False])